# Digit Recognizer Submission
The code starts with the imports and data handling/preprocessing.

In [ ]:
# Set-up code partially from "Deep Neural Network Keras Way" notebook

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
from keras import  backend as K
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers.experimental import preprocessing

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

# read train and test sets
train = pd.read_csv("../input/digit-recognizer/train.csv")
print(train.shape)
test= pd.read_csv("../input/digit-recognizer/test.csv")
print(test.shape)

# set up training and testing data
X_train = (train.iloc[:,1:].values).astype('float32') # all pixel values
y_train = train.iloc[:,0].values.astype('int32') # only labels i.e targets digits
X_test = test.values.astype('float32')

X_train = X_train.reshape(X_train.shape[0], 28, 28,1)
X_test = X_test.reshape(X_test.shape[0], 28, 28,1)

for i in range(6, 9):
    plt.subplot(330 + (i+1))
    plt.imshow(X_train[i], cmap=plt.get_cmap('gray'))
    plt.title(y_train[i])

In [ ]:
# Data Preprocessing (from the aforementioned notebook)
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)

def standardize(x): 
    return (x-mean_px)/std_px

from keras.utils.np_utils import to_categorical
y_train= to_categorical(y_train)
num_classes = y_train.shape[1]
num_classes

In [ ]:
# fix random seed for reproducibility
seed = 43
np.random.seed(seed)

# The Model
I am using a convolutional neural network with the following architecture:

1. InputLayer(28, 28, 1)
2. Preprocessing
3. Conv2D(kernel_size=3, filters=32, activation='relu')
4. BatchNormalization
5. Conv2D(kernel_size=3, filters=64, activation='relu')
6. BatchNormalization
7. Conv2D(kernel_size=5, filters=128, activation='relu')
8. BatchNormalization
9. Conv2D(kernel_size=5, filters=128, activation='relu')
10. MaxPool2D
11. BatchNormalization
12. Flatten
13. Dense(units=1024,activation='relu')
14. Dense(units=10, activation='softmax')

In [ ]:
# Function for the model
def my_model():
# Building the model
    model = keras.Sequential([
        layers.InputLayer(input_shape=[28,28,1]),
        preprocessing.RandomContrast(0.2),
        preprocessing.RandomTranslation(height_factor=0.1,width_factor=0.1),
        layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
        layers.BatchNormalization(axis=1),
        layers.Conv2D(filters=64, kernel_size=3, activation='relu'),
        layers.BatchNormalization(axis=1),
        layers.Conv2D(filters=128, kernel_size=5, activation='relu'),
        layers.BatchNormalization(axis=1),
        layers.Conv2D(filters=128, kernel_size=5, activation='relu'),
        layers.MaxPool2D(),
        layers.BatchNormalization(axis=1),
        layers.Flatten(),
        layers.Dense(units=1024,activation='relu'),
        layers.Dense(units=10, activation='softmax')
    ])

    # Compiling the model with 'adam' optimizer,
    # "categorical_crossentropy" loss, and "accuracy" metric
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
# prepares data for training
from sklearn.model_selection import train_test_split
X = X_train
y = y_train
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=1)

In [ ]:
# Fitting the model with Early Stopping
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
model = my_model()
history=model.fit(
    X_train, y_train,
    validation_data = (X_val, y_val),
    epochs=50,
    callbacks=[callback]
)

This model trains to a maximum validation accuracy of about 97%.

Now the model will be trained on the whole training data for final submission.

In [ ]:
# train final model with all of the training data
callback_final = keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1, restore_best_weights=True)
final_model= my_model()
history=final_model.fit(X, y, epochs=50, callbacks=[callback_final])

In [ ]:
# Final submission
predictions = model.predict(X_test, verbose=0)
predictions_final = predictions.argmax(axis=1)
print(predictions_final)

submissions=pd.DataFrame({"ImageId": list(range(1,len(predictions_final)+1)),
                         "Label": predictions_final})
submissions.to_csv("submission.csv", index=False, header=True)
print("Finished uploading submission file to csv")